In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily = TavilySearchResults()

In [5]:
result = tavily.invoke("what is large language model")

In [6]:
result

[{'title': 'Large language model - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/Large_language_model',
  'content': 'Related articlesGlossary of artificial intelligenceList of datasets for machine-learning researchList of datasets in computer vision and image processingOutline of machine learning\nvte\nA large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text. [...] tasks and understanding. The NTL Model outlines how specific neural structures of the human brain shape the nature of thought and language and in turn what are the computational properties of such neural systems that can be applied to model thought and language in a computer system. After a framework for modeling language in a computer systems was established, the focus shifted to establishing frameworks for computer sy

In [7]:
result[0]['content']

'Related articlesGlossary of artificial intelligenceList of datasets for machine-learning researchList of datasets in computer vision and image processingOutline of machine learning\nvte\nA large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text. [...] tasks and understanding. The NTL Model outlines how specific neural structures of the human brain shape the nature of thought and language and in turn what are the computational properties of such neural systems that can be applied to model thought and language in a computer system. After a framework for modeling language in a computer systems was established, the focus shifted to establishing frameworks for computer systems to generate language with acceptable grammar. In his 2014 book titled [...] Let \n\n\n\nx\n\n\n{\\displaystyle x}\n\n

In [8]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.tools import WikipediaQueryRun

In [9]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.utilities import WikipediaAPIWrapper

In [10]:
wiki_api = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=200)
wikipedia = WikipediaQueryRun(api_wrapper=wiki_api)

In [11]:
arxiv_api = ArxivAPIWrapper(top_k_results=3, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api)

In [12]:
tools = [wikipedia, arxiv, tavily]

In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='qwen-qwq-32b')

In [14]:
llm_with_tools = llm.bind_tools(tools=tools)

In [18]:
result = llm_with_tools.invoke("what is large language model?")

In [21]:
tool_calls = result.tool_calls
tool_calls

[{'name': 'wikipedia',
  'args': {'query': 'large language model'},
  'id': 'call_kxcc',
  'type': 'tool_call'}]

In [30]:
wikipedia.invoke(tool_calls[0]['args'])

'Page: Large language model\nSummary: A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language model'

In [51]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage

In [52]:
class State(TypedDict):
    messages : Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [53]:
class LLMNode:
    def __init__(self, llm):
        self.llm = llm
    def __call__(self, state:State):
        return {"messages": [self.llm.invoke(state['messages'])]}
    
llm_node = LLMNode(llm_with_tools)

In [54]:
from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tools)

In [55]:
graph_builder.add_node("llm", llm_node)
graph_builder.add_node("tools", tool_node)

In [56]:
from langgraph.graph import START
from langgraph.prebuilt import tools_condition

graph_builder.add_edge(START, "llm")
graph_builder.add_conditional_edges("llm", tools_condition)
graph_builder.add_edge("tools", "llm")

In [57]:
agent = graph_builder.compile()

In [59]:
from IPython.display import Image, display
display(Image(agent.get_graph().draw_mermaid_png()))

ValueError: Failed to render the graph using the Mermaid.INK API. Status code: 502.

In [64]:
messages = agent.invoke({"messages":HumanMessage(content="what is MCP in AI")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

what is MCP in AI
================================== Ai Message ==================================
Tool Calls:
  wikipedia (call_e6h2)
 Call ID: call_e6h2
  Args:
    query: MCP AI
================================= Tool Message =================================
Name: wikipedia

Page: Model Context Protocol
Summary: The Model Context Protocol (MCP) is an open standard developed by the artificial intelligence company Anthropic for enabling large language model (LLM) applicatio
================================== Ai Message ==================================

The Model Context Protocol (MCP) is an open standard developed by Anthropic (an artificial intelligence company) to enable interoperability and advanced capabilities in large language models (LLMs) and their applications. 

Key aspects of MCP include:
1. **Purpose**: It allows LLMs to better understand and generate structured data (e.g., forms, code, or 

In [81]:
messages['messages'][3].pretty_print()

================================== Ai Message ==================================

The Model Context Protocol (MCP) is an open standard developed by Anthropic (an artificial intelligence company) to enable interoperability and advanced capabilities in large language models (LLMs) and their applications. 

Key aspects of MCP include:
1. **Purpose**: It allows LLMs to better understand and generate structured data (e.g., forms, code, or tables) while maintaining context and safety.
2. **Features**: It provides a structured way to format inputs and outputs, enabling features like:
   - **Step-by-step reasoning**  
   - **Structured data handling**  
   - **Reduced prompt injection risks**  
3. **Adoption**: It is used in Anthropic’s Claude models and is designed for broader industry adoption as an open protocol.

MCP addresses challenges in how AI systems process complex, real-world tasks by standardizing communication between models and applications.
